<a href="https://colab.research.google.com/github/Laniqiu/dough/blob/dd/run_dd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install stanza
!pip install pycantonese
!pip install jiagu
!pip install jieba

In [6]:
import sys

sys.path.insert(0, "/content/drive/MyDrive/dough/dd")
sys.path.insert(0, "/content/drive/MyDrive/dough/")
sys.path.insert(0, "/content/drive/MyDrive/")


from dd_utils import *

In [ ]:
import logging
import pandas as pd
import numpy as np
from collections import OrderedDict
from pathlib import Path
import stanza
from stanza.models.common.doc import Document

logging.basicConfig(level=logging.INFO)

_root = "/content/drive/MyDrive/"
_p = Path(_root).joinpath("ddata")
files = _p.joinpath("posed").glob("*.txt")

def parsing_dd(files):
    """
    parsing and cal dd and save to original files
    """
    nlp_tra = stanza.Pipeline(lang="zh-hant", 
                            processors='depparse', depparse_pretagged=True)
    nlp_sim = stanza.Pipeline(lang="zh-hans", 
                            processors='depparse', depparse_pretagged=True)

    for f in files:
        logging.info(f.name)
        if "simp" in f.name:
            lang = "zh-hans"  
            nlp = nlp_sim
        else:
            lang =  "zh-hant"
            nlp = nlp_tra
        logging.info("lang:{}".format(lang))
        
        cols = ["sid", "wid", "text", "upos*", "xpos"]
        all_ = pd.read_table(f, sep="\t")
        data = all_[cols].values

        sents, word_id = [], []
        sent_id = list(set(data[:,0].tolist()))
        sent_id.sort()
        for sid in sent_id:  # sent id
            each_sen = []
            ixs = np.where(data[:,0] == sid)
            for idx, (w, p, x) in enumerate(data[:,2:][ixs]):
                each_w = {'id': idx + 1, 'text': w, 'lemma': w, 'upos': p, "xpos": x}
                each_sen.append(each_w)
            word_id.append(data[:,1][ixs].tolist())  # word id
            sents.append(each_sen)

        doc = nlp(Document(sents))
        logging.info("Done parsing")
        logging.info("Start calculating dd")

        # cal dd, 去除标点符号
        rt_id, rt_txt = find_roots(doc)
        roots = list(zip(rt_txt, rt_id))
        roots = list(map(str, roots))

        assert len(doc.sentences) == len(rt_id)  # 确保每一棵树都有root

        dep2root, dwp = depth2root(rt_id, doc, word_id)
        dis2root, dwr = ldd2root(rt_id, doc, word_id)
        dis2head, dwh = ldd2head(doc, word_id)

        dwri = [e[1] for l in dwr for e in l]
        dwhi = [e[1] for l in dwh for e in l]
        dwpi = [e[1] for l in dwp for e in l]

        # 规范格式输出  save to files
        all_["LD2ROOT"] = dwri
        all_["LD2HEAD"] = dwhi
        all_["DEPTH2ROOT"] = dwpi
        all_.to_csv(f, sep="\t") 
    